<a href="https://colab.research.google.com/github/1004516/SE-ALES-Y-SISTEMAS/blob/main/Solucion_Parcial_2/Simulacion__punto_1_Parcial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python punto1_fijo.py

In [ ]:
##simulacion punto 1 parcial 2 -------------------------------------------------
####Instalar librerias necesarias......
# ----------------------------------------------------
!pip install yt-dlp pydub soundfile matplotlib numpy

import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from pydub import AudioSegment
import yt_dlp
import os

# ----------------------------------------------------
# 1. DESCARGA AUTOMÁTICA (5 SEGUNDOS FIJOS)
# ----------------------------------------------------

YOUTUBE_URL = "https://www.youtube.com/watch?v=3oK22ll4cxw"

def download_fixed_youtube_audio(output_file="message.wav", duration_sec=5):
    """
    Descarga audio desde YouTube.
    Extrae exactamente duration_sec segundos.
    """
    print("Descargando audio desde YouTube...")

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'temp_audio.%(ext)s',
        'quiet': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([YOUTUBE_URL])

    # Encuentra archivo descargado
    for f in os.listdir():
        if f.startswith("temp_audio"):
            downloaded_file = f
            break

    print("Convirtiendo a WAV...")

    audio = AudioSegment.from_file(downloaded_file)
    audio = audio.set_channels(1)       # Mono
    audio = audio.set_frame_rate(44100) # fs = 44100 Hz
    audio = audio[:duration_sec * 1000] # EXACTAMENTE 5 segundos

    audio.export(output_file, format="wav")
    os.remove(downloaded_file)

    print("Audio listo como", output_file)


# ----------------------------------------------------
# 2. FFT Utilities
# ----------------------------------------------------

def fft_mag(x, fs):
    N = len(x)
    X = np.fft.fftshift(np.fft.fft(x))
    f = np.fft.fftshift(np.fft.fftfreq(N, 1/fs))
    return f, np.abs(X) / N

def apply_lpf_fft(x, fs, fcut):
    N = len(x)
    X = np.fft.fft(x)
    freqs = np.fft.fftfreq(N, 1/fs)
    mask = np.abs(freqs) <= fcut
    X_filtered = X * mask
    return np.real(np.fft.ifft(X_filtered))


# ----------------------------------------------------
# 3. MODULACIÓN + DEMODULACIÓN
# ----------------------------------------------------

def simulate_dsb_demod(audio_file="message.wav", A1=1.0, f0=15000, fcut=5000):

    m, fs = sf.read(audio_file)
    if m.ndim > 1:
        m = m[:,0]
    N = len(m)
    t = np.arange(N) / fs

    # Portadora
    carrier = np.cos(2*np.pi*f0*t)

    # (1) Modulación DSB-SC
    x = A1 * m * carrier

    # (2) Mezclador
    lo = np.cos(2*np.pi*f0*t)
    y = x * lo

    # (3) LPF usando FFT
    y_lpf = apply_lpf_fft(y, fs, fcut)

    # (4) Escalado
    recovered = (2/A1) * y_lpf

    # FFTs
    f_m, Mmag = fft_mag(m, fs)
    f_x, Xmag = fft_mag(x, fs)
    f_y, Ymag = fft_mag(y, fs)
    f_yl, Ylmag = fft_mag(y_lpf, fs)
    f_r, Rmag = fft_mag(recovered, fs)

    # ----------------------------------------------------
    # Gráficas en tiempo
    # ----------------------------------------------------
    plt.figure(figsize=(12,10))
    plt.subplot(5,1,1); plt.plot(t, m); plt.title("m(t) — mensaje")
    plt.subplot(5,1,2); plt.plot(t, x); plt.title("x(t) — señal modulada")
    plt.subplot(5,1,3); plt.plot(t, y); plt.title("y(t) — salida del mezclador")
    plt.subplot(5,1,4); plt.plot(t, y_lpf); plt.title("y_LPF(t) — salida del LPF (FFT)")
    plt.subplot(5,1,5); plt.plot(t, recovered); plt.title("Señal recuperada m_rec(t)")
    plt.tight_layout()
    plt.show()

    # ----------------------------------------------------
    # Gráficas en frecuencia
    # ----------------------------------------------------
    plt.figure(figsize=(12,10))
    plt.subplot(5,1,1); plt.plot(f_m, Mmag); plt.title("M(f) — espectro del mensaje")
    plt.subplot(5,1,2); plt.plot(f_x, Xmag); plt.title("X(f) — espectro modulada")
    plt.subplot(5,1,3); plt.plot(f_y, Ymag); plt.title("Y(f) — post-mezclador")
    plt.subplot(5,1,4); plt.plot(f_yl, Ylmag); plt.title("Y_LPF(f)")
    plt.subplot(5,1,5); plt.plot(f_r, Rmag); plt.title("M_rec(f)")
    plt.tight_layout()
    plt.show()


# ----------------------------------------------------
# 4. EJECUCIÓN AUTOMÁTICA
# ----------------------------------------------------

if __name__ == "__main__":
    download_fixed_youtube_audio()
    simulate_dsb_demod()
